In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
#from config import g_key
from citipy import citipy
import csv as atlantic_working

In [3]:
# Import API key
from config import weather_api_key

In [4]:
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [8]:
awdf = pd.read_csv('atlantic working.csv')
awdf['Latitude'].head()
awdf['Longitude'].head()

0    94.8W
1    95.4W
2    96.0W
3    96.5W
4    96.8W
Name: Longitude, dtype: object

In [9]:
#set "city center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)

In [10]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [11]:
awdf['Latitude']=df_city['lat']
awdf['Longitude']=df_city['lng']
df_city.count()

City    6500
lat     6500
lng     6500
dtype: int64

In [12]:
awdf.head()

,ID,Name,Year,Month,Date,Hour,Event,Status,Latitude,Longitude,...,A low that is neither of the 3 cyclone variantsw Wind SW,A low that is neither of the 3 cyclone variantsw Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,1851,6,25,0,,Hurricane Intensity > 64 knoTropical Storm 34...,30.029916,-98.456461,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,1851,6,25,600,,Hurricane Intensity > 64 knoTropical Storm 34...,29.713713,-95.953724,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,1851,6,25,1200,,Hurricane Intensity > 64 knoTropical Storm 34...,29.188570,-97.359426,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,1851,6,25,1800,,Hurricane Intensity > 64 knoTropical Storm 34...,31.290828,-94.969133,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,1851,6,25,2100,Landfall,Hurricane Intensity > 64 knoTropical Storm 34...,29.468151,-98.047205,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [13]:
#check length of cities
len(list(df_city["City"].unique()))

112

In [14]:
cities = list(df_city["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

target_cities

['canyon+lake',
 'katy',
 'victoria',
 'lufkin',
 'seguin',
 'killeen',
 'brenham',
 'jacksonville',
 'huntsville',
 'taylor',
 'live+oak',
 'brownwood',
 'san+angelo',
 'copperas+cove',
 'pflugerville',
 'palestine',
 'round+rock',
 'orange',
 'kerrville',
 'natchitoches',
 'lumberton',
 'bryan',
 'temple',
 'galena+park',
 'gatesville',
 'lakeway',
 'liberty',
 'bay+city',
 'lockhart',
 'el+campo',
 'mission+bend',
 'del+rio',
 'bellmead',
 'alvin',
 'robinson',
 'fredericksburg',
 'uvalde',
 'kyle',
 'conroe',
 'galveston',
 'leander',
 'san+marcos',
 'lake+jackson',
 'sulphur',
 'angleton',
 'beaumont',
 'hondo',
 'college+station',
 'corsicana',
 'austin',
 'atascocita',
 'wharton',
 'stephenville',
 'port+arthur',
 'rosenberg',
 'humble',
 'harker+heights',
 'freeport',
 'aldine',
 'belton',
 'baytown',
 'san+antonio',
 'nacogdoches',
 'channelview',
 'vidor',
 'eagle+pass',
 'spring',
 'pleasanton',
 'new+braunfels',
 'tomball',
 'the+woodlands',
 'houston',
 'beeville',
 'ander

In [15]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

In [17]:
# base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
# temp=[]
# wind_speed=[]
# pressure=[]
# city_name=[]

# for city in target_cities[:4]:
#     query_url=f"{base_current_url}appid={weather_api_key}&q={city},US&units=imperial"
#     req = requests.get(query_url)
#     response1=req.json()
#     latitude=response1["coord"]["lat"]
#     longitude=response1["coord"]["lon"]

#     for x in unix_time():
#         historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
#         hist_json = requests.get(historical_url).json()
#         temp.append(hist_json["current"]["temp"])
#         pressure.append(hist_json["current"]["pressure"])
#         wind_speed.append(hist_json["current"]["wind_speed"])
#         city_name.append(response1['name'])
#         print(historical_url)

In [18]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng

def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    temp=[]
    wind_speed=[]
    pressure=[]
    city_name=[]
    country_code=[]
    for city in target_cities[:5]:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},Texas,USA&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]
#             print(response1)

        except:
            print("City not found. Skipping...")
            pass
        if response1["sys"]["country"]=="US":
            for x in unix_time():
                historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
                hist_json = requests.get(historical_url).json()
                temp.append(hist_json["current"]["temp"])
                pressure.append(hist_json["current"]["pressure"])
                wind_speed.append(hist_json["current"]["wind_speed"])
                city_name.append(response1['name'])
                country_code.append(response1["sys"]["country"])
    d=dict()
    d["City Name"] = city_name
    d["Country"] = country_code
    d["Temperature"] = temp
    d["Wind_Speed"] = wind_speed
    d["Pressure"] = pressure    
    return(d)
        #print ("--"*30)

In [19]:
df2 =pd.DataFrame(find_location(target_cities))

In [20]:
df2

,City Name,Country,Temperature,Wind_Speed,Pressure
0,Canyon Lake,US,64.08,1.01,1012
1,Canyon Lake,US,65.46,7.00,1010
2,Canyon Lake,US,70.39,3.00,1009
3,Canyon Lake,US,79.02,1.01,1008
4,Canyon Lake,US,71.92,0.00,1010
5,Katy,US,81.46,1.99,1010
6,Katy,US,81.27,1.99,1008
7,Katy,US,82.18,1.01,1008
8,Katy,US,82.04,0.00,1010
9,Katy,US,78.35,1.01,1013


In [ ]:
df[df["Country"]=="US"]